In [1]:
pip install pysimplegui


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xlrd==1.2.0

Note: you may need to restart the kernel to use updated packages.


In [3]:
import PySimpleGUI as sg
import pickle
import pandas as pd
import os.path
from os import path

In [5]:
# Choose a Theme for the Layout
sg.theme('LightGrey1')
  
layout = [[sg.T("         "), sg.Radio('Mass Input', "RADIO1", enable_events=True, default=False, key="mass_input")],
          [sg.T("         "), sg.Radio('Single Input', "RADIO1", enable_events=True, default=True, key="single_input")],
          [sg.Text('Enter Ticket Description', key="_ticket_desc_lbl_"), sg.Multiline(key="_ticket_desc_input_")],
          [sg.Text('Description column in above file', key="_ticket_desc_col_lbl_"), sg.InputText(key="_ticket_desc__col_input_")],
          [sg.Button('Predict')],
          [sg.Text('Your ticket will be assigned to group', key="_ticket_group_lbl_"), sg.InputText(key="_ticket_group_input_")],
          [sg.Text('                                                                                        ', key="_error_lbl_")],
          
         ]
  
###Setting Window
window = sg.Window('AI Enabled Ticketing Tool', layout, size=(600,600)).finalize()
window.Element('_ticket_desc__col_input_').Update(disabled=True)

###Showing the Application, also GUI functions can be placed here.
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event=="Exit":
        break
    elif event == 'mass_input':
        print("Mass input")
        window.Element('_ticket_desc__col_input_').Update(disabled=False)
        window.Element('_ticket_desc_lbl_').Update('Enter File name') 
        
    elif event == 'single_input':
        print("Single input")
        window.Element('_ticket_desc_lbl_').Update('Enter Ticket Description')
        window.Element('_ticket_desc__col_input_').Update(disabled=True)
        
    elif event == 'Predict':
        text_predict = []
        text_predict.clear()    
        
        if values['mass_input'] == True:
            description = values['_ticket_desc_input_']
            userInput = description.splitlines()
            print(description)
            window.Element('_ticket_group_input_').Update('')
            
            #check the extension
            extension = description.split(sep=".", maxsplit=1)
            print(extension)
            if len(extension) != 2:
                window.Element('_error_lbl_').Update(background_color='#FFCCCB')
                window.Element('_error_lbl_').Update('Please enter the file extension')
            elif ( ( len(extension) == 2 )  and ( extension[1] == 'xlsx\n' or extension[1] == 'xls\n' ) ) :
                print("extension available as excel")
                # Fetch the data from input data stored in Goolge drive
                print(userInput)
                filename = userInput[0]
                
                if path.exists(filename) == True:
                    incidents_data = pd.read_excel(filename)
                    text_predict = incidents_data['Ticket_Description']
                    if len(text_predict) > 0:
                        with open('SVC Classifier - 2.pkl', 'rb') as f:
                          classifier = pickle.load(f)

                        #Predict
                        predicted_group = classifier.predict(text_predict)
                        print(predicted_group)

                        predicted_groups = pd.DataFrame(columns=['Ticket Descriptions', 'Assignment Groups'])
                        predicted_groups['Ticket Descriptions'] = text_predict
                        predicted_groups['Assignment Groups'] = predicted_group
                        with open('Assignment-Group-Predictions.pkl', 'wb') as f:
                          pickle.dump(predicted_groups, f)
                          print('File pickled')
                        window.Element('_error_lbl_').Update(background_color='#00D100')
                        window.Element('_error_lbl_').Update('Prediction/s completed')
                        #print(predicted_groups)

                        window.Element('_ticket_group_input_').Update('Updated in the file named Assignment-Group-Predictions.pkl stored in same directory')
                else:
                    window.Element('_error_lbl_').Update(background_color='#FFCCCB')
                    window.Element('_error_lbl_').Update('File does not exist')
            elif ( ( len(extension) == 2 )  and (extension[1] == 'csv\n') ):
                print("extension available as csv")
                # Fetch the data from input data stored in Goolge drive
                filename = userInput[0]
                if path.exists(filename) == True:
                    incidents_data = pd.read_csv(filename)
                    text_predict = incidents_data['Ticket_Description']
                    if len(text_predict) > 0:
                        with open('SVC Classifier - 2.pkl', 'rb') as f:
                          classifier = pickle.load(f)

                        #Predict
                        predicted_group = classifier.predict(text_predict)
                        print(predicted_group)

                        predicted_groups = pd.DataFrame(columns=['Ticket Descriptions', 'Assignment Groups'])
                        predicted_groups['Ticket Descriptions'] = text_predict
                        predicted_groups['Assignment Groups'] = predicted_group
                        with open('Assignment-Group-Predictions.pkl', 'wb') as f:
                          pickle.dump(predicted_groups, f)
                          print('File pickled')
                        window.Element('_error_lbl_').Update(background_color='#00D100')
                        window.Element('_error_lbl_').Update('Prediction/s completed')
                        #print(predicted_groups)

                        window.Element('_ticket_group_input_').Update('Updated in the file named Assignment-Group-Predictions.pkl stored in same directory')
                else:
                    window.Element('_error_lbl_').Update(background_color='#FFCCCB')
                    window.Element('_error_lbl_').Update('File does not exist')
            elif len(extension) == 2:
                print("extension available not as excel")
                window.Element('_error_lbl_').Update(background_color='#FFCCCB')
                window.Element('_error_lbl_').Update('Only excel or csv files are supported')
            
        elif values['single_input'] == True: 
            text_predict.append(values["_ticket_desc_input_"])
            
            #Fetch the pickled model
            if len(text_predict) > 0:
                # The pickled file is added in the same gitHub Repo named SVC Classifier - 2.pkl
                with open('SVC Classifier - 2.pkl', 'rb') as f:
                  classifier = pickle.load(f)

                #Predict
                predicted_group = classifier.predict(text_predict)
                
                window.Element('_ticket_group_input_').Update('GRP_'+str(predicted_group[0]))
                window.Element('_error_lbl_').Update(background_color='#00D100')
                window.Element('_error_lbl_').Update('Prediction/s completed')
    
window.close()

In [5]:
with open('Assignment-Group-Predictions.pkl', 'rb') as f:
    predicted_group_df = pickle.load(f)

In [6]:
predicted_group_df

,Ticket Descriptions,Assignment Groups
0,received from: monitoring_tool@company.com ...,45
1,received from: monitoring_tool@company.com ...,45
2,reset password,0
